<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=47229bfdcc129ae8f6dec9bb8b1f0177d4a83c7ecde9a08823589721fc32ad3c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 15:18:18
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: -1.65 L (-1.14%)
Current PnL: -21.70 L (-14.22%)
CY Booked + Current PnL: -9.05 L (-5.93%)
-------------------
Total profit:  2.06 L
Total loss:  -23.77 L
-------------------
Total Booked + Current PnL: 17.93 L (14.27%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.85%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.69 L (62.08%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.53%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-8.03,64.0,X-LC,5.75,230584.0,20218.0,8555.0,0.34,9.61,3.71,13.68,37.0,2.36,1.63,27.72,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.76,174439.0,33599.0,10885.0,1.24,23.86,6.24,31.59,66.0,3.09,1.23,56.66,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.42,53.0,M-SC,1.76,87600.0,-13177.0,13271.0,-0.01,-13.08,15.15,0.09,245.0,-0.99,0.62,15.09,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.14,60.0,X-MC,5.58,149487.0,13354.0,17236.0,-0.98,9.81,11.53,22.47,91.0,0.77,1.05,24.06,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-12.67,32.0,X-MC,2.70,86355.0,-4323.0,19231.0,-1.96,-4.77,22.27,16.44,101.0,-0.22,0.61,14.45,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-8.88,70.0,X-LC,4.64,230006.0,-21762.0,74982.0,5.11,-8.64,32.60,21.13,27.0,-0.29,1.62,23.72,X40,ATH,PAINTS
5,ANGELONE,3033.00,20.92,68.0,X-SC,5.84,214266.0,23260.0,34433.0,3.76,12.18,16.07,30.21,157.0,0.68,1.51,35.60,X40N,NTT,FINANCE
14,BSOFT,836.99,-14.02,59.0,H-SC,11.31,96620.0,-44033.0,116814.0,2.30,-31.31,120.90,51.74,131.0,-0.38,0.68,11.79,XR,ATH,IT
61,SAIL,228.00,53.97,65.0,M-MC,10.10,243319.0,18357.0,149301.0,2.26,8.16,61.36,74.53,192.0,0.12,1.72,43.05,XY24,BTT,STEEL
68,SONACOMS,806.63,-30.74,70.0,M-SC,6.88,86152.0,-15007.0,55008.0,1.76,-14.83,63.85,39.54,202.0,-0.27,0.61,21.48,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3278.38,31.0,M-SC,3.38,151674.0,-45458.0,135293.0,-6.64,-23.06,89.20,45.57,236.0,-0.34,1.07,7.53,XY24,NTT,AUTO
70,SURYODAY,240.00,60.09,46.0,H-SC,15.65,146268.0,-32803.0,98526.0,-5.70,-18.32,67.36,36.71,135.0,-0.33,1.03,45.13,XR,NTT,BANKS
46,KPIGREEN,731.05,6.33,52.0,H-SC,7.27,126580.0,1283.0,57645.0,-5.59,1.02,45.54,47.03,141.0,0.02,0.89,58.49,MH,ATH,POWER
59,REPCOHOME,880.00,-53.80,60.0,H-SC,7.14,262804.0,-23701.0,281043.0,-5.54,-8.27,106.94,89.82,134.0,-0.08,1.85,39.04,XY24,NTT,FINANCE
17,CERA,9475.00,-24.58,42.0,H-SC,1.91,137655.0,-38248.0,80267.0,-4.92,-21.74,58.31,23.89,149.0,-0.48,0.97,18.88,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,26.6,47.0,M-LC,10.51,206582.0,1506.0,111058.0,0.33,0.73,53.76,54.89,52.0,0.01,1.46,25.01,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.42,53.0,M-SC,1.76,87600.0,-13177.0,13271.0,-0.01,-13.08,15.15,0.09,245.0,-0.99,0.62,15.09,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,45.0,H-SC,2.68,221580.0,-48087.0,84422.0,-0.94,-17.83,38.10,13.47,138.0,-0.57,1.56,12.67,XY24,NTT,PAINTS
17,CERA,9475.0,-24.58,42.0,H-SC,1.91,137655.0,-38248.0,80267.0,-4.92,-21.74,58.31,23.89,149.0,-0.48,0.97,18.88,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,63.0,M-SC,1.52,97888.0,-29917.0,29905.0,0.03,-23.41,30.55,-0.01,232.0,-1.00,0.69,10.84,OX40N,NTT,AC
67,SIS,528.0,1990.67,38.0,H-SC,2.77,84380.0,-26652.0,50257.0,-0.03,-24.00,59.56,21.26,156.0,-0.53,0.60,14.01,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,69.0,M-LC,10.76,174439.0,33599.0,10885.0,1.24,23.86,6.24,31.59,66.0,3.09,1.23,56.66,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,53.97,65.0,M-MC,10.10,243319.0,18357.0,149301.0,2.26,8.16,61.36,74.53,192.0,0.12,1.72,43.05,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,47.0,M-LC,10.51,206582.0,1506.0,111058.0,0.33,0.73,53.76,54.89,52.0,0.01,1.46,25.01,X5K,ATH,METALS
46,KPIGREEN,731.05,6.33,52.0,H-SC,7.27,126580.0,1283.0,57645.0,-5.59,1.02,45.54,47.03,141.0,0.02,0.89,58.49,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,69.0,M-LC,10.76,174439.0,33599.0,10885.0,1.24,23.86,6.24,31.59,66.0,3.09,1.23,56.66,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,5.32,123390.0,8592.0,85448.0,-2.42,7.48,69.25,81.92,223.0,0.10,0.87,46.07,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,5.32,123390.0,8592.0,85448.0,-2.42,7.48,69.25,81.92,223.0,0.10,0.87,46.07,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,52.0,H-SC,5.84,129384.0,6048.0,125580.0,-1.09,4.90,97.06,106.72,119.0,0.05,0.91,29.43,AR,ATH,MISC
46,KPIGREEN,731.05,6.33,52.0,H-SC,7.27,126580.0,1283.0,57645.0,-5.59,1.02,45.54,47.03,141.0,0.02,0.89,58.49,MH,ATH,POWER
31,HINDZINC,730.22,26.60,47.0,M-LC,10.51,206582.0,1506.0,111058.0,0.33,0.73,53.76,54.89,52.0,0.01,1.46,25.01,X5K,ATH,METALS
86,WIPRO,420.00,-16.52,37.0,M-LC,5.55,146549.0,-4396.0,113854.0,-0.65,-2.91,77.69,72.51,53.0,-0.04,1.03,2.94,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,21.0,X-LC,28.48,147560.0,-128963.0,239032.0,-2.52,-46.64,161.99,39.81,54.0,-0.54,1.04,4.00,XY24,NTT,AUTO
44,JSWINFRA,342.00,-24.78,25.0,X-MC,4.66,188696.0,-11651.0,42155.0,-3.50,-5.82,22.34,15.23,178.0,-0.28,1.33,24.56,X40N,NTT,REALTY
23,DMART,5391.45,-21.15,26.0,X-LC,8.25,200845.0,-9562.0,68729.0,-3.96,-4.54,34.22,28.12,38.0,-0.14,1.42,18.01,X40N,ATH,FMCG
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.88,88511.0,-40159.0,85457.0,0.03,-31.21,96.55,35.20,219.0,-0.47,0.62,1.59,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-13.94,30.0,X-LC,6.84,241580.0,-15483.0,50611.0,-1.22,-6.02,20.95,13.67,24.0,-0.31,1.70,13.16,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.60,34.0,X-MC,0.89,197895.0,-2925.0,70728.0,-1.55,-1.46,35.74,33.76,80.0,-0.04,1.40,3.66,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.14,197654.0,-2484.0,23382.0,-1.15,-1.24,11.83,10.44,4.0,-0.11,1.39,4.25,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,38.0,H-LC,1.21,151930.0,-34165.0,81647.0,-2.60,-18.36,53.74,25.51,15.0,-0.42,1.07,11.96,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.73,35.0,X-MC,3.08,304437.0,-23554.0,130086.0,-2.47,-7.18,42.73,32.48,92.0,-0.18,2.15,3.42,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.09,66.0,X-LC,4.11,246029.0,11885.0,36732.0,-1.41,5.08,14.93,20.77,86.0,0.32,1.73,12.35,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,34.0,X-SC,37.72,49700.0,-15306.0,166237.0,-1.27,-23.55,334.48,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79620.0,-3890.0,23918.0,-0.81,-4.66,30.04,23.98,82.0,-0.16,0.56,0.89,X40,ATH,APPARELS
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.88,88511.0,-40159.0,85457.0,0.03,-31.21,96.55,35.20,219.0,-0.47,0.62,1.59,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,-7.18,40.0,X-MC,7.35,216930.0,-46435.0,155756.0,-0.98,-17.63,71.80,41.51,84.0,-0.30,1.53,1.80,XY25,ATH,FMCG
1,ABB,7934.00,-42.40,36.0,H-LC,5.26,240912.0,-20707.0,139922.0,-4.29,-7.92,58.08,45.57,7.0,-0.15,1.70,2.51,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-49.86,34.0,X-SC,37.72,49700.0,-15306.0,166237.0,-1.27,-23.55,334.48,232.18,198.0,-0.09,0.35,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.48,33.0,X-MC,7.24,79620.0,-3890.0,23918.0,-0.81,-4.66,30.04,23.98,82.0,-0.16,0.56,0.89,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,41.0,X-SC,4.32,81178.0,-63982.0,139910.0,-0.32,-44.08,172.35,52.31,136.0,-0.46,0.57,8.66,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-12.67,32.0,X-MC,2.70,86355.0,-4323.0,19231.0,-1.96,-4.77,22.27,16.44,101.0,-0.22,0.61,14.45,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.87,27.0,X-SC,6.88,88511.0,-40159.0,85457.0,0.03,-31.21,96.55,35.20,219.0,-0.47,0.62,1.59,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.33,46.0,X-LC,9.88,281370.0,-64586.0,131287.0,0.10,-18.67,46.66,19.28,1.0,-0.49,1.98,4.02,X40,ATH,IT
39,INFY,2275.00,-19.24,51.0,X-LC,5.12,314601.0,1713.0,169979.0,0.62,0.55,54.03,54.87,3.0,0.01,2.22,8.88,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.14,197654.0,-2484.0,23382.0,-1.15,-1.24,11.83,10.44,4.0,-0.11,1.39,4.25,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,6.89,300051.0,-15791.0,128452.0,-0.15,-5.00,42.81,35.67,5.0,-0.12,2.12,8.16,X40N,ATH,FMCG
1,ABB,7934.00,-42.40,36.0,H-LC,5.26,240912.0,-20707.0,139922.0,-4.29,-7.92,58.08,45.57,7.0,-0.15,1.70,2.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,35.0,L-SC,27.74,74765.0,-38784.0,78832.0,-3.78,-34.16,105.44,35.27,268.0,-0.49,0.53,82.99,XR,NTT,HOTELS
7,ASIANTILES,137.00,7383.33,65.0,L-SC,10.59,82116.0,-11694.0,88308.0,-1.70,-12.47,107.54,81.67,269.0,-0.13,0.58,59.02,XR,NTT,CERAMICS
49,MASFIN,398.61,-18.70,50.0,H-SC,8.37,92250.0,-5730.0,27334.0,-1.69,-5.85,29.63,22.05,152.0,-0.21,0.65,34.09,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,5.32,123390.0,8592.0,85448.0,-2.42,7.48,69.25,81.92,223.0,0.10,0.87,46.07,XR,NTT,DURABLES
46,KPIGREEN,731.05,6.33,52.0,H-SC,7.27,126580.0,1283.0,57645.0,-5.59,1.02,45.54,47.03,141.0,0.02,0.89,58.49,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,73.00,63.0,H-SC,2.28,149996.0,-32779.0,143846.0,-3.13,-17.93,95.90,60.77,125.0,-0.23,1.06,34.35,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-30.74,70.0,M-SC,6.88,86152.0,-15007.0,55008.0,1.76,-14.83,63.85,39.54,202.0,-0.27,0.61,21.48,AR,ATH,AUTO
46,KPIGREEN,731.05,6.33,52.0,H-SC,7.27,126580.0,1283.0,57645.0,-5.59,1.02,45.54,47.03,141.0,0.02,0.89,58.49,MH,ATH,POWER
5,ANGELONE,3033.00,20.92,68.0,X-SC,5.84,214266.0,23260.0,34433.0,3.76,12.18,16.07,30.21,157.0,0.68,1.51,35.60,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7383.33,65.0,L-SC,10.59,82116.0,-11694.0,88308.0,-1.70,-12.47,107.54,81.67,269.0,-0.13,0.58,59.02,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.42
2,50,75.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.86
LC    33.10
MC    23.06
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.05
X40      16.26
X40N     12.48
XY25     11.93
XR        9.85
AR        7.78
OX40N     6.51
X200      1.77
X5K       1.46
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.19
X-LC    22.44
X-MC    18.71
M-SC    12.64
X-SC     5.58
M-LC     5.14
H-LC     4.54
H-MC     2.31
M-MC     1.72
L-SC     1.45
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.26,-4.67,38.51
IT,12.43,-19.25,83.14
FINANCE,12.22,-9.14,58.62
MISC,7.78,-17.58,76.89
ELECTRICAL,5.68,-12.19,51.97
PAINTS,5.58,-16.72,34.12
BANKS,4.03,-18.07,86.76
INSURANCE,3.92,0.53,36.55
AUTO,2.72,-49.15,111.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3315328.0,22
XR,1258655.0,13
AR,1184519.0,9
X40,849750.0,12
X40N,698726.0,9
OX40N,561877.0,9
XY25,483268.0,8
SR,274565.0,2
X5K,111058.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3400267.0,24
M-SC,1253648.0,15
X-MC,1223235.0,14
X-LC,1097389.0,13
X-SC,556310.0,6
H-LC,294873.0,3
M-LC,279966.0,4
H-MC,259781.0,2
L-SC,256595.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1191601.0      6
           AR         840134.0      5
           XR         785930.0      7
M-SC       XY24       723397.0      6
X-MC       XY24       575556.0      4
X-LC       X40        463744.0      5
           XY24       314151.0      2
X-SC       X40N       304616.0      4
X-MC       X40        300549.0      6
H-SC       SR         274565.0      2
           OX40N      250392.0      3
X-LC       X40N       233913.0      3
H-LC       AR         221569.0      2
X-MC       XY25       186933.0      2
H-MC       XY24       184200.0      1
L-SC       XR         167140.0      2
X-SC       XY24       166237.0      1
X-MC       X40N       160197.0      2
M-MC       XY24       149301.0      1
M-SC       OX40N      146449.0      4
           XR         134578.0      2
           XY25       126408.0      1
           AR         122816.0      2
M-LC       XR         113854.0      1
           X5K        111058.0      1
L-SC       OX40N       89455.0      1
X-LC       XY25        85581.0      3
X-SC       X40         85457.0      1
H-MC       OX40N       75581.0      1
H-LC       X200        73304.0      1
H-SC       MH          57645.0      1
L-MC       XR          57153.0      1
M-LC       XY25        44169.0      1
L-LC       XY25        40177.0      1
M-LC       XY24        10885.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
